<a href="https://colab.research.google.com/github/dwwatson1/coronavirus_pandemic_playbook/blob/michael_work/maryland_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [62.2 kB]
Hit:13 http://ppa.launchpad.net/cran

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://cdc-main-database.s3.amazonaws.com/COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv"), header=True)

In [4]:
df.show()

+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+-------------------------------+-------------------+--------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|case_month|res_state|state_fips_code|res_county|county_fips_code|     age_group|   sex|   race|ethnicity|case_positive_specimen_interval|case_onset_interval|             process|exposure_yn|      current_status|symptom_status|hosp_yn| icu_yn|death_yn|underlying_conditions_yn|
+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+-------------------------------+-------------------+--------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|   2020-08|       KY|             21|   GRAYSON|           21085|  0 - 17 years|    NA|     NA|       NA|                           null|                  0|        

In [5]:
# Drop unnecessary columns
df = df.drop("case_positive_specimen_interval", "case_onset_interval", "process")

In [6]:
df.show()

+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|case_month|res_state|state_fips_code|res_county|county_fips_code|     age_group|   sex|   race|ethnicity|exposure_yn|      current_status|symptom_status|hosp_yn| icu_yn|death_yn|underlying_conditions_yn|
+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|   2020-08|       KY|             21|   GRAYSON|           21085|  0 - 17 years|    NA|     NA|       NA|    Missing|Laboratory-confir...|   Symptomatic|     No|Missing|      No|                    null|
|   2021-01|       NC|             37|    MARTIN|           37117|  0 - 17 years|    NA|     NA|       NA|    Unknown|Laboratory-confir...|  Asymptomatic|     No|Unknown|      No| 

In [8]:
# Filter dates between March 2020 and December 2020
df = df.filter((df.case_month >= '2020-03') & (df.case_month <= '2020-12'))

In [9]:
df.show()

+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|case_month|res_state|state_fips_code|res_county|county_fips_code|     age_group|   sex|   race|ethnicity|exposure_yn|      current_status|symptom_status|hosp_yn| icu_yn|death_yn|underlying_conditions_yn|
+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|   2020-08|       KY|             21|   GRAYSON|           21085|  0 - 17 years|    NA|     NA|       NA|    Missing|Laboratory-confir...|   Symptomatic|     No|Missing|      No|                    null|
|   2020-09|       GA|             13|   CATOOSA|           13047|  0 - 17 years|Female|Unknown|  Missing|    Missing|Laboratory-confir...|       Missing|Missing|Missing| Missing| 

In [10]:
# Filter for Maryland
df = df.filter((df.res_state == 'MD'))

In [11]:
df.show()

+----------+---------+---------------+--------------+----------------+--------------+------+--------------+-------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|case_month|res_state|state_fips_code|    res_county|county_fips_code|     age_group|   sex|          race|          ethnicity|exposure_yn|      current_status|symptom_status|hosp_yn| icu_yn|death_yn|underlying_conditions_yn|
+----------+---------+---------------+--------------+----------------+--------------+------+--------------+-------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|   2020-07|       MD|             24|    DORCHESTER|           24019|18 to 49 years|Female|         White|    Hispanic/Latino|    Missing|Laboratory-confir...|   Symptomatic|    Yes|Missing|      No|                     Yes|
|   2020-09|       MD|             24|  QUEEN ANNE'S|           24035|18 to 49 years|Female|    

In [12]:
df.toPandas().to_csv('maryland_data.csv')